<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [3]:
import pandas as pd
import psycopg2

In [4]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [5]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [6]:
# текст запроса
query_3_1 = f'''select 
           count(id) "количество вакансий"
           from vacancies
           
        '''

In [7]:
df = pd.read_sql_query(query_3_1, connection)
df

/tmp/ipykernel_17933/3851344990.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


количество вакансий
0                49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [8]:
query_3_2 = f'''select 
           count(distinct id) "количество работодателей"
           from employers
           
        '''

In [9]:
df = pd.read_sql_query(query_3_2, connection)
df

/tmp/ipykernel_17933/3588252056.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_2, connection)


количество работодателей
0                     23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [10]:
query_3_3 = f'''select 
           count(distinct id) "количество регионов"
           from areas
           
        '''

In [11]:
df = pd.read_sql_query(query_3_3, connection)
df

/tmp/ipykernel_17933/2018224393.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_3, connection)


количество регионов
0                 1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [12]:
query_3_4 = f'''select 
           count(distinct id) "сферы деятельности"
           from industries
           
        '''

In [13]:
df = pd.read_sql_query(query_3_4, connection)
df

/tmp/ipykernel_17933/2172479966.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_4, connection)


сферы деятельности
0                 294

***

In [14]:
# выводы по предварительному анализу данных

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [15]:
query_4_1 = f'''select 
           a.name area,
           count(*) cnt
           from vacancies v join areas a on v.area_id = a.id 
           group by a.id
           order by cnt desc 
           
        '''

In [16]:
df = pd.read_sql_query(query_4_1, connection)
df.head(5)

/tmp/ipykernel_17933/1677473053.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1, connection)


area   cnt
0           Москва  5333
1  Санкт-Петербург  2851
2            Минск  2112
3      Новосибирск  2006
4           Алматы  1892

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [17]:
query_4_2 = f'''select 
           count(id) "вакансии с з/п"
           from vacancies
           where salary_from is not null or salary_to is not null
           
        '''

In [18]:
df = pd.read_sql_query(query_4_2, connection)
df

/tmp/ipykernel_17933/2168503709.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_2, connection)


вакансии с з/п
0           24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [19]:
query_4_3 = f'''select
          'средняя для нижней границы' "показатель", 
           round(AVG(salary_from)) "значение"
           from vacancies
           union
           select
          'средняя для верхней границы' , 
           round(AVG(salary_to))
           from vacancies
        '''

In [20]:
df = pd.read_sql_query(query_4_3, connection)
df

/tmp/ipykernel_17933/2222372203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_3, connection)


показатель  значение
0  средняя для верхней границы  110537.0
1   средняя для нижней границы   71065.0

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [21]:
query_4_4 = f'''select 
           schedule "рабочий график",
           employment "тип трудоустройства",
           count(*) cnt
           from vacancies
           group by schedule, employment 
           order by cnt desc 
           
        '''

In [22]:
df = pd.read_sql_query(query_4_4, connection)
df.head(2)

/tmp/ipykernel_17933/1120173435.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_4, connection)


рабочий график тип трудоустройства    cnt
0       Полный день    Полная занятость  35367
1  Удаленная работа    Полная занятость   7802

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [23]:
query_4_5 = f'''select 
           experience "опыт работы",
           count(*) cnt
           from vacancies
           group by experience 
           order by cnt  
           
        '''

In [24]:
df = pd.read_sql_query(query_4_5, connection)
df

/tmp/ipykernel_17933/2601166264.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_5, connection)


опыт работы    cnt
0         Более 6 лет   1337
1           Нет опыта   7197
2       От 3 до 6 лет  14511
3  От 1 года до 3 лет  26152

***

In [25]:
# выводы по детальному анализу вакансий

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [26]:
query_5_1 = f'''select 
           e.name "название компании",
           count(v.id) cnt
           from employers e join vacancies v on v.employer_id = e.id	
           group by e.name 
           order by cnt desc
           
        '''

In [27]:
df = pd.read_sql_query(query_5_1, connection)
df.head(5)

/tmp/ipykernel_17933/437255355.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_1, connection)


название компании   cnt
0            Яндекс  1933
1        Ростелеком   491
2          Тинькофф   444
3              СБЕР   428
4     Газпром нефть   331

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [28]:
query_5_2 = f'''select 
             a.name "название региона",
             count(v.id) "кол-во вакансий",
             count(distinct e.id) "кол-во работадателей"
   	         from areas a
 	         left join employers e on a.id = e.area
 	         left join vacancies v on v.area_id = a.id
             group by a.id
             having count(v.id) = 0 and count(distinct e.id) != 0
             order by 3 desc
             '''

In [29]:
df = pd.read_sql_query(query_5_2, connection)
df.head(5)

/tmp/ipykernel_17933/976017165.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_2, connection)


название региона  кол-во вакансий  кол-во работадателей
0              Россия                0                   410
1           Казахстан                0                   207
2  Московская область                0                    75
3  Краснодарский край                0                    19
4            Беларусь                0                    18

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [56]:
query_5_3 = f'''select 
                emp.name as name_emp,
                count(distinct vac.area_id) as cnt_area_vac
                from public.employers as emp
                join public.vacancies as vac on vac.area_id = emp.area
                group by 1
                order by 2 desc
         '''

In [57]:
df = pd.read_sql_query(query_5_3, connection)
df.head()

/tmp/ipykernel_17933/1332990265.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_3, connection)


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [32]:
# текст запроса

In [33]:
# результат запроса

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [34]:
# текст запроса

In [35]:
# результат запроса

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [36]:
# текст запроса

In [37]:
# результат запроса

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [38]:
# код для получения списка городов-милионников

In [39]:
# текст запроса

In [40]:
# результат запроса

***

In [41]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [42]:
# текст запроса

In [43]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [44]:
# текст запроса

In [45]:
# результат запроса

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [46]:
# текст запроса

In [47]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [48]:
# текст запроса

In [49]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [50]:
# текст запроса

In [51]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [52]:
# текст запроса

In [53]:
# результат запроса

***

In [54]:
# выводы по предметному анализу

# Общий вывод по проекту

In [55]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования